In [1]:
from pathlib import Path
import tensorflow as tf

SEED = 42
IMG_SIZE = (28, 28)
BATCH = 64

root = Path("data")
train_root = root / "train"
test_root  = root / "test"

# train / val (train에서 20%를 검증으로)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_root, validation_split=0.2, subset="training", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="int"
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_root, validation_split=0.2, subset="validation", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="int"
)

# test는 train의 class_names 순서를 그대로 사용해서 라벨 정렬 문제 예방
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_root, image_size=IMG_SIZE, batch_size=BATCH, shuffle=False, label_mode="int",
    class_names=train_ds.class_names
)

print("classes:", train_ds.class_names)
num_classes = len(train_ds.class_names)

Found 900 files belonging to 3 classes.
Using 720 files for training.
Found 900 files belonging to 3 classes.
Using 180 files for validation.
Found 150 files belonging to 3 classes.
classes: ['paper', 'rock', 'scissor']


In [2]:
from tensorflow import keras
from tensorflow.keras import layers

keras.backend.clear_session()

inputs = keras.Input(shape=IMG_SIZE + (3,))  
x = layers.RandomFlip("horizontal")(inputs)
x = layers.RandomRotation(0.08)(x)
x = layers.RandomZoom(0.10)(x)
x = layers.RandomContrast(0.10)(x)
x = layers.Rescaling(1./255)(x)

# Block 1
x = layers.Conv2D(48, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(48, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)
x = layers.Dropout(0.25)(x)

# Block 2
x = layers.Conv2D(96, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(96, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)
x = layers.Dropout(0.30)(x)

# Block 3
x = layers.Conv2D(160, 3, padding="same", activation="relu")(x)
x = layers.BatchNormalization()(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(96, activation="relu")(x)
x = layers.Dropout(0.35)(x)

outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 random_flip (RandomFlip)    (None, 28, 28, 3)         0         
                                                                 
 random_rotation (RandomRot  (None, 28, 28, 3)         0         
 ation)                                                          
                                                                 
 random_zoom (RandomZoom)    (None, 28, 28, 3)         0         
                                                                 
 random_contrast (RandomCon  (None, 28, 28, 3)         0         
 trast)                                                          
                                                                 
 rescaling (Rescaling)       (None, 28, 28, 3)         0     

In [3]:
ckpt = keras.callbacks.ModelCheckpoint(filepath="best.weights.h5", monitor="val_accuracy", mode="max",
                                       save_best_only=True, save_weights_only=True)

plateau = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                            patience=3, min_lr=1e-5, verbose=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[ckpt, plateau], verbose=1)

Epoch 1/50
12/12 [==============================] - 5s 237ms/step - loss: 0.8373 - accuracy: 0.6014 - val_loss: 1.1005 - val_accuracy: 0.3389 - lr: 0.0010
Epoch 2/50
12/12 [==============================] - 3s 204ms/step - loss: 0.4793 - accuracy: 0.7931 - val_loss: 1.1334 - val_accuracy: 0.3167 - lr: 0.0010
Epoch 3/50
12/12 [==============================] - 3s 207ms/step - loss: 0.3486 - accuracy: 0.8694 - val_loss: 1.2170 - val_accuracy: 0.3167 - lr: 0.0010
Epoch 4/50
12/12 [==============================] - ETA: 0s - loss: 0.3248 - accuracy: 0.8722
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
12/12 [==============================] - 3s 205ms/step - loss: 0.3248 - accuracy: 0.8722 - val_loss: 1.6621 - val_accuracy: 0.3167 - lr: 0.0010
Epoch 5/50
12/12 [==============================] - 3s 219ms/step - loss: 0.2254 - accuracy: 0.9181 - val_loss: 2.6499 - val_accuracy: 0.3167 - lr: 5.0000e-04
Epoch 6/50
12/12 [==============================] - 3s 212ms/s

In [4]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

test_opt = test_ds.cache()

# 예측
y_true = np.concatenate([y.numpy() for _, y in test_opt.unbatch().batch(4096)])
y_pred = model.predict(test_opt, verbose=0).argmax(axis=1)

present = np.array(sorted(np.unique(y_true)))
target_names = [train_ds.class_names[i] for i in present]

# Confusion matrix
print("\nConfusion matrix (labels in order:", present, ")")
print(confusion_matrix(y_true, y_pred, labels=present))

# Classification report
print("\nClassification report:")
print(classification_report(y_true, y_pred, labels=present, target_names=target_names, digits=4))

# loss / accuracy
test_loss, test_acc = model.evaluate(test_opt, verbose=0)
print(f"\n[Test] loss={test_loss:.4f}  acc={test_acc:.4f}")


Confusion matrix (labels in order: [0 1 2] )
[[50  0  0]
 [22 28  0]
 [50  0  0]]

Classification report:
              precision    recall  f1-score   support

       paper     0.4098    1.0000    0.5814        50
        rock     1.0000    0.5600    0.7179        50
     scissor     0.0000    0.0000    0.0000        50

    accuracy                         0.5200       150
   macro avg     0.4699    0.5200    0.4331       150
weighted avg     0.4699    0.5200    0.4331       150


[Test] loss=3.3746  acc=0.5200


c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

---

### 2nd 도전

In [5]:
# 소량 샘플로 과적합 테스트
small_train = train_ds.unbatch().take(60).batch(32)   
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
hist_small = model.fit(small_train, epochs=30, verbose=0)

print("small-set max acc:", max(hist_small.history["accuracy"]))

small-set max acc: 1.0


In [6]:
from collections import Counter

# 학습률 낮추기
model.compile(optimizer=keras.optimizers.Adam(3e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# 클래스 가중치
from collections import Counter
cnt = Counter()
for _, yb in train_ds.unbatch():
    cnt[int(yb.numpy())] += 1
K = len(train_ds.class_names); total = sum(cnt.values())
class_weight = {i: total/(K*cnt.get(i,1)) for i in range(K)}
print("class_weight:", class_weight)

# 체크포인트(가중치만 저장) + Plateau
ckpt = keras.callbacks.ModelCheckpoint("best.weights.h5", monitor="val_accuracy",
                                       mode="max", save_best_only=True, save_weights_only=True)
plateau = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                            patience=3, min_lr=1e-5, verbose=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=50,
                    callbacks=[ckpt, plateau], class_weight=class_weight, verbose=1)

model.load_weights("best.weights.h5")

class_weight: {0: 1.00418410041841, 1: 1.0084033613445378, 2: 0.9876543209876543}
Epoch 1/50
12/12 [==============================] - 5s 226ms/step - loss: 0.1024 - accuracy: 0.9625 - val_loss: 1.0864 - val_accuracy: 0.7389 - lr: 3.0000e-04
Epoch 2/50
12/12 [==============================] - 3s 204ms/step - loss: 0.0730 - accuracy: 0.9819 - val_loss: 1.0975 - val_accuracy: 0.7611 - lr: 3.0000e-04
Epoch 3/50
12/12 [==============================] - 3s 218ms/step - loss: 0.0602 - accuracy: 0.9833 - val_loss: 0.5536 - val_accuracy: 0.8444 - lr: 3.0000e-04
Epoch 4/50
12/12 [==============================] - 3s 207ms/step - loss: 0.0777 - accuracy: 0.9694 - val_loss: 1.0497 - val_accuracy: 0.6722 - lr: 3.0000e-04
Epoch 5/50
12/12 [==============================] - 3s 200ms/step - loss: 0.0510 - accuracy: 0.9806 - val_loss: 1.9270 - val_accuracy: 0.5889 - lr: 3.0000e-04
Epoch 6/50
12/12 [==============================] - ETA: 0s - loss: 0.0566 - accuracy: 0.9819
Epoch 6: ReduceLROnPlateau re

In [7]:
test_opt = test_ds.cache()

y_true = np.concatenate([y.numpy() for _, y in test_opt.unbatch().batch(4096)])
y_pred = model.predict(test_opt, verbose=0).argmax(axis=1)
present = np.array(sorted(np.unique(y_true)))
target_names = [train_ds.class_names[i] for i in present]

print("\nConfusion matrix (labels in order:", present, ")")
print(confusion_matrix(y_true, y_pred, labels=present))

print("\nClassification report:")
print(classification_report(y_true, y_pred, labels=present, target_names=target_names, digits=4))

test_loss, test_acc = model.evaluate(test_opt, verbose=0)
print(f"\n[Test] loss={test_loss:.4f}  acc={test_acc:.4f}")


Confusion matrix (labels in order: [0 1 2] )
[[31 19  0]
 [ 0 50  0]
 [19 31  0]]

Classification report:
              precision    recall  f1-score   support

       paper     0.6200    0.6200    0.6200        50
        rock     0.5000    1.0000    0.6667        50
     scissor     0.0000    0.0000    0.0000        50

    accuracy                         0.5400       150
   macro avg     0.3733    0.5400    0.4289       150
weighted avg     0.3733    0.5400    0.4289       150


[Test] loss=2.5470  acc=0.5400


c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\envs\ds_study\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

---

### 3rd 도전

In [8]:
IMG_SIZE = (160, 160)  

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_root, validation_split=0.2, subset="training", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="int")
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_root, validation_split=0.2, subset="validation", seed=SEED,
    image_size=IMG_SIZE, batch_size=BATCH, label_mode="int")
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_root, image_size=IMG_SIZE, batch_size=BATCH,
    shuffle=False, label_mode="int", class_names=train_ds.class_names)

print("classes:", train_ds.class_names)

Found 900 files belonging to 3 classes.
Using 720 files for training.
Found 900 files belonging to 3 classes.
Using 180 files for validation.
Found 150 files belonging to 3 classes.
classes: ['paper', 'rock', 'scissor']


In [9]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

keras.backend.clear_session()

num_classes = len(train_ds.class_names)

augment = keras.Sequential([layers.RandomFlip("horizontal"),
                            layers.RandomRotation(0.05),
                            layers.RandomZoom(0.08),
                            layers.RandomContrast(0.05)], name="augment")

base = EfficientNetB0(include_top=False, weights="imagenet", input_shape=IMG_SIZE + (3,))
base.trainable = False   

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = augment(inputs)
x = preprocess_input(x)             
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

ckpt = keras.callbacks.ModelCheckpoint("best.weights.h5", monitor="val_accuracy",
                                       mode="max", save_best_only=True, save_weights_only=True)
plateau = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                            patience=3, min_lr=1e-5, verbose=1)

history = model.fit(train_ds, validation_data=val_ds, epochs=10,
                    callbacks=[ckpt, plateau], verbose=1)

Epoch 1/10
12/12 [==============================] - 21s 1s/step - loss: 0.8506 - accuracy: 0.6542 - val_loss: 0.5795 - val_accuracy: 0.9056 - lr: 0.0010
Epoch 2/10
12/12 [==============================] - 14s 1s/step - loss: 0.4943 - accuracy: 0.9194 - val_loss: 0.3850 - val_accuracy: 0.9556 - lr: 0.0010
Epoch 3/10
12/12 [==============================] - 14s 1s/step - loss: 0.3334 - accuracy: 0.9625 - val_loss: 0.2899 - val_accuracy: 0.9667 - lr: 0.0010
Epoch 4/10
12/12 [==============================] - 13s 1s/step - loss: 0.2548 - accuracy: 0.9708 - val_loss: 0.2348 - val_accuracy: 0.9667 - lr: 0.0010
Epoch 5/10
12/12 [==============================] - 14s 1s/step - loss: 0.2124 - accuracy: 0.9722 - val_loss: 0.2001 - val_accuracy: 0.9722 - lr: 0.0010
Epoch 6/10
12/12 [==============================] - 13s 1s/step - loss: 0.1735 - accuracy: 0.9806 - val_loss: 0.1753 - val_accuracy: 0.9722 - lr: 0.0010
Epoch 7/10
12/12 [==============================] - 13s 1s/step - loss: 0.1511 - a

In [10]:
test_opt = test_ds.cache()
y_true = np.concatenate([y.numpy() for _, y in test_opt.unbatch().batch(4096)])
y_pred = model.predict(test_opt, verbose=0).argmax(axis=1)
present = np.array(sorted(np.unique(y_true)))
target_names = [train_ds.class_names[i] for i in present]

print("\nConfusion matrix (labels in order:", present, ")")
print(confusion_matrix(y_true, y_pred, labels=present))

print("\nClassification report:")
print(classification_report(y_true, y_pred, labels=present,
                            target_names=target_names, digits=4))

test_loss, test_acc = model.evaluate(test_opt, verbose=0)
print(f"\n[Test] loss={test_loss:.4f}  acc={test_acc:.4f}")


Confusion matrix (labels in order: [0 1 2] )
[[50  0  0]
 [ 0 50  0]
 [ 1  0 49]]

Classification report:
              precision    recall  f1-score   support

       paper     0.9804    1.0000    0.9901        50
        rock     1.0000    1.0000    1.0000        50
     scissor     1.0000    0.9800    0.9899        50

    accuracy                         0.9933       150
   macro avg     0.9935    0.9933    0.9933       150
weighted avg     0.9935    0.9933    0.9933       150


[Test] loss=0.1293  acc=0.9933
